In [11]:
from init import *
from math import nan

# Vanilla option

In [12]:
Call = 0
Put = 1

In [13]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 1000
model.NumPaths = 100000
model.Add(sdk.IndependentGaussian())
underlying = model.Add(sdk.GeometricalBrownianMotion(
    start = 130,
    drift = 0.0,
    diffusion = 0.3
))

option = model.Add(sdk.Option(
    underlying = underlying.GetStateNumber(),
    call_put = sdk.Option.Put,
    strike = 140
))


In [14]:
model.evaluations.append(sdk.EvaluationPoint(0,2))
model.json()

'{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "GeometricalBrownianMotion", "refs": [], "args": [0.0, 0.3], "start": 130}, {"name": "Option", "refs": [0], "args": [140, 1], "start": 0}], "evaluations": [{"state": 0, "time": 2}]}'

In [15]:
results = run (model)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,GeometricalBrownianMotion,,0,2.0,1000,100000,129.665894,0.182262,57.636356,1.435091
1,Option,,1,2.0,1000,100000,28.207945,0.090784,28.708279,0.621633


## MC option price vs strike

In [16]:
def vanilla_option_price(spot,strike,volatility,expiry,call_put,mc_paths=10000,mc_steps=1000,seed=-1):
    model = sdk.Model()
    model.TimeStart = 0
    model.TimeSteps = mc_steps
    model.NumPaths = mc_paths
    model.RandomSeed = seed

    model.Add(sdk.IndependentGaussian())
    underlying_process = model.Add(sdk.GeometricalBrownianMotion(spot,0,volatility))

    option_process = model.Add(sdk.Updater(
        name = "Option",
        args = [strike,call_put],
        refs = [underlying_process.GetStateNumber()],
        start = 0
    ))
    
    model.evaluations.append(sdk.EvaluationPoint(0,expiry))

    results = run(model)
        
    return results.GetStateEvaluationResult(option_process.GetStateNumber(),0).mean

In [17]:
import plotly.express as px
import numpy as np

In [18]:
f = lambda x: vanilla_option_price(130,x,0.3,10,Call,mc_paths=1000)
vx = np.linspace(50,250,20)

fig = px.scatter (
    title = f'Call Option price vs strike',
    x = vx,
    y = [f(x) for x in vx]
)
fig.show()

In [19]:
f = lambda x: vanilla_option_price(130,x,0.3,10,Call,mc_paths=1000,seed=0)
vx = np.linspace(50,250,20)

fig = px.scatter (
    title = f'Call Option price vs strike',
    x = vx,
    y = [f(x) for x in vx]
)
fig.show()

# Add barrier

In [20]:
def single_barrier_option_price (
    spot = 130,
    strike = 140,
    volatility = 0.3,
    barrier1_level = 150,
    mc_paths = 10000,
    mc_steps = 1000,
    seed = -1
):
    # prepare model
    model = sdk.Model()
    model.TimeStart = 0
    model.TimeSteps = mc_steps
    model.NumPaths = mc_paths
    model.RandomSeed = seed
    model.Add(sdk.IndependentGaussian())
    # Add a basic 'underlying' process
    underlying_process = model.Add(sdk.GeometricalBrownianMotion(
        start = spot,
        drift = 0.0,
        diffusion = volatility,
        title = 'Underlying'
    ))

    put_option_process = model.Add(sdk.Option(
        underlying = underlying_process.GetStateNumber(),
        strike = strike,
        call_put = sdk.Option.Put,
        title = f'Put, K={strike}'
    ))

    call_option_process = model.Add(sdk.Option(
        underlying = underlying_process.GetStateNumber(),
        strike = strike,
        call_put = sdk.Option.Call,
        title = f'Call, K={strike}'
    ))
    
    UpAndOutDigitalBarrier_process = model.Add(sdk.Barrier(
        underlying = underlying_process.GetStateNumber(),
        start = 1,
        level = barrier1_level,
        direction = sdk.Barrier.DirectionUp,
        action = sdk.Barrier.ActionSet,
        value = 0,
        title = f'Digital:UpAndOut level={barrier1_level}'
    ))

    UpAndInDigitalBarrier_process = model.Add(sdk.Barrier(
        underlying = underlying_process.GetStateNumber(),
        start = 0,
        level = barrier1_level,
        direction = sdk.Barrier.DirectionUp,
        action = sdk.Barrier.ActionSet,
        value = 1,
        title = f'Digital:UpAndIn level={barrier1_level}'
    ))
    
    UpAndOutPutBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            put_option_process.GetStateNumber(),
            UpAndOutDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndOutPut level={barrier1_level}'
    ))
    
    UpAndInPutBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            put_option_process.GetStateNumber(),
            UpAndInDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndInPut level={barrier1_level}'
    ))

    UpAndOutCallBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            call_option_process.GetStateNumber(),
            UpAndOutDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndOutCall level={barrier1_level}'
    ))
    
    UpAndInCallBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            call_option_process.GetStateNumber(),
            UpAndInDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndInCall level={barrier1_level}'
    ))

    return model

In [21]:
model_1b = single_barrier_option_price(
    volatility = 0.1,
    spot       = 130,
    strike     = 140,
    mc_steps   = 1000,
    mc_paths   = 10000,
    seed       = 111
)
model_1b.evaluations.append(sdk.EvaluationPoint(0,10))
results_1b = run (model_1b)
results_1b.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,GeometricalBrownianMotion,Underlying,0,10.0,1000,10000,129.842957,0.419588,41.958755,0.949827
1,Option,"Put, K=140",1,10.0,1000,10000,22.497404,0.235357,23.535675,0.738102
2,Option,"Call, K=140",2,10.0,1000,10000,12.340369,0.255216,25.521635,2.879818
3,Barrier,Digital:UpAndOut level=150,3,10.0,1000,10000,0.413300,0.004924,0.492426,0.352134
4,Barrier,Digital:UpAndIn level=150,4,10.0,1000,10000,0.586700,0.004924,0.492426,-0.352135
5,Multiplication,UpAndOutPut level=150,5,10.0,1000,10000,16.598116,0.239209,23.920944,1.162817
6,Multiplication,UpAndInPut level=150,6,10.0,1000,10000,5.899290,0.133248,13.324811,2.563810
7,Multiplication,UpAndOutCall level=150,7,10.0,1000,10000,0.032908,0.004061,0.406120,14.831571
8,Multiplication,UpAndInCall level=150,8,10.0,1000,10000,12.307462,0.255343,25.534271,2.878817


### Run the same model, but ask to compute the state at several time points

In [22]:
model_2b = single_barrier_option_price(
    volatility = 0.1,
    spot       = 130,
    strike     = 140,
    mc_steps   = 1000,
    mc_paths   = 10000,
    seed       = 111
)
model_2b.evaluations.append(sdk.EvaluationPoint(0,2))
model_2b.evaluations.append(sdk.EvaluationPoint(0,5))
model_2b.evaluations.append(sdk.EvaluationPoint(0,10))
results_2b = run (model_2b)
results_2b.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,GeometricalBrownianMotion,Underlying,0,2.0,200,10000,130.009369,0.186210,18.620951,0.485307
1,Option,"Put, K=140",1,2.0,200,10000,13.696061,0.130750,13.075028,0.677474
2,Option,"Call, K=140",2,2.0,200,10000,3.705437,0.086187,8.618731,3.085286
3,Barrier,Digital:UpAndOut level=150,3,2.0,200,10000,0.728500,0.004447,0.444733,-1.027583
4,Barrier,Digital:UpAndIn level=150,4,2.0,200,10000,0.271500,0.004447,0.444733,1.027582
5,Multiplication,UpAndOutPut level=150,5,2.0,200,10000,13.262281,0.132786,13.278645,0.698063
6,Multiplication,UpAndInPut level=150,6,2.0,200,10000,0.433782,0.024779,2.477859,7.710246
7,Multiplication,UpAndOutCall level=150,7,2.0,200,10000,0.192966,0.009685,0.968457,5.902811
8,Multiplication,UpAndInCall level=150,8,2.0,200,10000,3.512471,0.086429,8.642926,3.121760
9,GeometricalBrownianMotion,Underlying,0,5.0,500,10000,129.835327,0.292846,29.284615,0.648456
